In [1]:
import os, glob
import numpy as np
import pandas as pd
from scipy.stats import norm

# Closed Form for Up-and-Out Barrier Call and Lookback Option

In [2]:
def delta_p(s, tau=.5, r=0.01, sigma=0.2):
    return (np.log(s) + (r+0.5*sigma**2)*tau) / sigma / np.sqrt(tau)

def delta_m(s, tau=.5, r=0.01, sigma=0.2):
    return (np.log(s) + (r-0.5*sigma**2)*tau) / sigma / np.sqrt(tau)

def up_and_out_call(S, K, B, tau=0.5, r=0.01, sigma=0.2):
    return (
        S*(S<B)*(
            norm.cdf(delta_p(S/K)) - 
            norm.cdf(delta_p(S/B)) -
            ((B/S)**(1+2*r/(sigma**2)))*(norm.cdf(delta_p((B**2)/K/S))-norm.cdf(delta_p(B/S)))
        ) - 
        np.exp(-r*tau)*K*(S<B)*(
            norm.cdf(delta_m(S/K)) - 
            norm.cdf(delta_m(S/B)) -
            ((S/B)**(1-2*r/(sigma**2)))*(norm.cdf(delta_m((B**2)/K/S))-norm.cdf(delta_m(B/S)))
        )
    )*(K<=B)

def lookback_option(S, option_type, tau=.5, r=0.01, sigma=0.2):
    return ((
        S*np.exp(-r*tau)*norm.cdf(-delta_m(1)) +
        S*(1+sigma**2/2/r)*norm.cdf(delta_p(1)) -
        S*np.exp(-r*tau)*(sigma**2/2/r)*norm.cdf(-delta_m(1)) -
        S
    ) if option_type == 'put' else
    (
        S*norm.cdf(delta_p(1)) -
        S*np.exp(-r*tau)*norm.cdf(delta_m(1)) +
        S*np.exp(-r*tau)*(sigma**2/2/r)*norm.cdf(delta_m(1))-
        S*(sigma**2/2/r)*norm.cdf(-delta_p(1))
    ))

# Statistics from Log Files

In [3]:
DATA_DIR = '.'
barrier_files = ['main_barrier.csv', 'signature_barrier.csv', 'regression_barrier.csv', 'regression_barrier_nvr.csv', 'galerkin_barrier.csv']
lookback_files = ['main_lookback.csv', 'signature_lookback.csv']

In [4]:

barrier_df_lst = []
for f in barrier_files:
    df = pd.read_csv(os.path.join(DATA_DIR, f))
    df['script'] = f.replace('.csv', '')
    barrier_df_lst.append(df)

barrier_df = pd.concat(barrier_df_lst)[['script', 'K', 'B', 'runtime', 'y0']]
barrier_df['ref_value'] = barrier_df.apply(lambda row: up_and_out_call(50, row['K'], row['B']), axis=1)
barrier_df = barrier_df[['script', 'K', 'B', 'ref_value', 'runtime', 'y0']]
barrier_stats = (
    barrier_df
    .groupby(['script', 'K', 'B'])
    .agg({'y0': ['mean', 'std'], 'ref_value': 'first', 'runtime': 'sum'})
    .reset_index()
)
barrier_stats['l1_error'] = np.abs(barrier_stats['ref_value']['first'] - barrier_stats['y0']['mean'])
barrier_stats.columns = ['script', 'K', 'B', 'mean', 'std', 'ref_value', 'runtime', 'l1_error']
barrier_stats = barrier_stats[['script', 'K', 'B', 'mean', 'std', 'ref_value', 'l1_error', 'runtime']]
barrier_stats
#print(barrier_stats.style.hide(axis="index").to_latex())

,script,K,B,mean,std,ref_value,l1_error,runtime
0,galerkin_barrier,47.0,46.0,0.004009,0.007099,0.00000,0.004009,1403.025542
1,galerkin_barrier,47.0,53.0,0.304883,0.035827,0.19058,0.114303,1440.547542
2,galerkin_barrier,52.0,46.0,0.000370,0.006270,0.00000,0.000370,1470.497430
3,galerkin_barrier,52.0,53.0,0.002776,0.010655,0.00089,0.001886,1509.128498
4,main_barrier,47.0,46.0,0.000018,0.000212,0.00000,0.000018,3545.296457
5,main_barrier,47.0,53.0,0.186287,0.081424,0.19058,0.004292,3574.632458
6,main_barrier,52.0,46.0,-0.000072,0.000233,0.00000,0.000072,3583.712391
7,main_barrier,52.0,53.0,0.003187,0.000684,0.00089,0.002297,3583.548356
8,regression_barrier,47.0,46.0,0.000000,0.000000,0.00000,0.000000,46.546981
9,regression_barrier,47.0,53.0,0.282338,0.006825,0.19058,0.091758,46.476965


In [5]:
lookback_df_lst = []
for f in lookback_files:
    df = pd.read_csv(os.path.join(DATA_DIR, f))
    df['script'] = f.replace('.csv', '')
    lookback_df_lst.append(df)

lookback_df = pd.concat(lookback_df_lst)#[['script', 'option_type', 'runtime', 'y0']]

lookback_df['ref_value'] = lookback_df.apply(lambda row: lookback_option(50, row['option_type']), axis=1)


lookback_stats = (
    lookback_df
    .groupby(['script', 'option_type'])
    .agg({'y0': ['mean', 'std'], 'ref_value': 'first', 'runtime': 'sum'})
    .reset_index()
)

lookback_stats['l1_error'] = np.abs(lookback_stats['ref_value']['first'] - lookback_stats['y0']['mean'])
lookback_stats.columns = ['script', 'option_type', 'mean', 'std', 'ref_value', 'runtime', 'l1_error']
lookback_stats = lookback_stats[['script', 'option_type', 'mean', 'std', 'ref_value', 'l1_error', 'runtime']]
lookback_stats
#print(lookback_stats.style.hide(axis="index").to_latex())

,script,option_type,mean,std,ref_value,l1_error,runtime
0,main_lookback,call,6.270066,1.491514,5.508991,0.761076,3596.572396
1,main_lookback,put,7.012987,1.608399,5.758367,1.254621,3605.677254
2,signature_lookback,call,1.794135,0.517429,5.508991,3.714856,116.055863
3,signature_lookback,put,3.063032,0.496111,5.758367,2.695335,115.974764
